# 3D U-Net implementation for Binary Segmentation task

 ### Create the four pair of images and masks tif files from the initial h5df file

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import torch

##you need to download the hdf5 data set into the same folder as this Colab Notebook 

#filename of the data set
filename = "datasets.hdf5"

#opening hdf5 file
f = h5py.File(filename, "r")

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/masks #read masks from folder /masks

In [ ]:
#reading masks
for key in list(f['time_lapse_train']['gt']):
    data = data = np.array(f['time_lapse_train']['gt'][key], dtype=np.int16)
    image_name = key.split('.')[0] + "_mask.tif"
    tifffile.imsave(image_name, data)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/images # Read images from folder /images

In [ ]:
#reading images
for key in list(f['time_lapse_train']['img']):
    data = data = np.array(f['time_lapse_train']['img'][key], dtype=np.int16)
    image_name = key.split('.')[0] + "_image.tif"
    tifffile.imsave(image_name, data)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/ # Back to the main folder

## Building the 3D UNet model

In [1]:
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate, LeakyReLU

def conv_block(input, num_filters):
    """Convolution block that goes down the U shape of the UNet model.
    Args:
        input:         array input X_train of 3D dimension (D,H,W), where D is the depth, H the height and W the width
        num_filters:   number of filters of the convolution, a scalar
    Returns:
        x:             convolved output, an array of 3D dimension (D,H,W)
    """
    x = Conv3D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = Conv3D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    return x

def encoder_block(input, num_filters):
    """Encoder block: convolution block followed by maxpooling.
    Args:
        input:         array input X_train of 3D dimension (D,H,W), where D is the depth, H the height and W the width
        num_filters:   number of filters of the convolution, a scalar
    Returns:
        x:             convolved input, an array of 3D dimension (D,H,W)
        p:             encoded input, an array of 3D dimension (D,H,W)
    """
    x = conv_block(input, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    """Decoder block: apply transposed convolution and concatenate the input with the skip features.
    Args:
        input:         array input X_train of 3D dimension (D,H,W), where D is the depth, H the height and W the width
        skip_features: array input of 3D dimension (D,H,W), this is the input from the encoder for concatenation
        num_filters:   number of filters of the convolution, a scalar
    Returns:
        x:             decoded input, an array of 3D dimension (D,H,W)
    """
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    """Building the UNet model using the encoder and decoder blocks. 
    Args:
        input_shape:   shape of the input array, tuple (D,H,W) when D is the depth, H the height and W the width
        n_classes:     number of input classes, a scalar
    Returns:
        model:         3D UNet model 
    """
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
        activation = 'sigmoid'
    else:
        activation = 'softmax'

    outputs = Conv3D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

### Installation of patchify library

In [2]:
#Use patchify to break large volumes into smaller for training 
#and also to put patches back together after prediction.
!pip install patchify

### GPU availability 

In [4]:
#Make sure the GPU is available. 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

### Loading the four pairs of images and masks tif files

In [4]:
from skimage import io
from patchify import patchify, unpatchify
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [13]:
#Load input images and masks. 
#Here we load 180x1024x1024 pixel volume. We will break it into patches of 16x128x128 for training, with a separation step of 5x128x128.
#This means that there is an overlap between the first patch and the next one of 5 in the depth dimension. 

#IMPORTANT: you need to change the file paths in case you do not upload the tif files in the same folder.. 
image = []

image.append(io.imread('/images/wt_pom1D_01_07_R3D_REF_image.tif'))
image.append(io.imread('/images/wt_pom1D_01_15_R3D_REF_image.tif'))
image.append(io.imread('/images/wt_pom1D_01_20_R3D_REF_image.tif'))
image.append(io.imread('/images/train/wt_pom1D_01_30_R3D_REF_image.tif'))

img_patches = []
img_patches.append(patchify(image[0], (16, 128, 128), step=(5, 128, 128)))
img_patches.append(patchify(image[1], (16, 128, 128), step=(5, 128, 128))) 
img_patches.append(patchify(image[2], (16, 128, 128), step=(5, 128, 128)))  
img_patches.append(patchify(image[3], (16, 128, 128), step=(5, 128, 128)))  

In [ ]:
mask = []

mask.append(io.imread('/masks/wt_pom1D_01_07_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_15_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_20_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_30_R3D_REF_mask.tif'))

mask_patches = []
mask_patches.append(patchify(mask[0], (16, 128, 128), step=(5, 128, 128)))
mask_patches.append(patchify(mask[1], (16, 128, 128), step=(5, 64, 64)))  
mask_patches.append(patchify(mask[2], (16, 128, 128), step=(5, 128, 128)))  
mask_patches.append(patchify(mask[3], (16, 128, 128), step=(5, 128, 128)))  

## Data pre processing

### Reshape the inputs

In [ ]:
#We reshape the patches to get an input image and input mask of shape (N, D, H, W), where N is the total number of patches, 
# D is the depth size of the patches, H is the height size of the patches, and W is the width size of the patches. 
input_img = np.reshape(img_patches[0], (-1, img_patches[0].shape[3], img_patches[0].shape[4], img_patches[0].shape[5]))
input_mask = np.reshape(mask_patches[0], (-1, mask_patches[0].shape[3], mask_patches[0].shape[4], mask_patches[0].shape[5]))

for i in range(1, 4):
    input_img += np.reshape(img_patches[i], (-1, img_patches[i].shape[3], img_patches[i].shape[4], img_patches[i].shape[5]))
    input_mask += np.reshape(mask_patches[i], (-1, mask_patches[i].shape[3], mask_patches[i].shape[4], mask_patches[i].shape[5]))

input_img = np.array(input_img)
input_mask = np.array(input_mask)

print(input_img.shape)
print(input_mask.shape)

### Removing empy patches 

In [ ]:
#Keep in a variable all the indices where the patches are empty in the whole sequences 
idx_img = np.where(input_mask.mean(axis=(1,2,3)) != 0)[0]

input_img = input_img[idx_img]
input_mask = input_mask[idx_img]

#Print the number of patches we have 
print(input_img.shape[0])
print(input_mask.shape[0])

In [22]:
#Standardize the input array of pixels by the maximum value the pixels
train_img = input_img / input_img.max() 

#Expand the dimension of the training sets by 1 to match with the input of the model (i.e. chanel number). 
train_img = np.expand_dims(train_img, axis=4)
train_mask = np.expand_dims(input_mask, axis=4)

#Since we are performing a binary segmentation, we need to binarize the masks. 
train_mask[train_mask>1] = 1

#Finally, we perform one hot encoding with the function to_categorical with a chosen number of classes of 2. 
n_classes=2
train_mask_cat = to_categorical(train_mask, num_classes=n_classes)

### Split randomly into training and validation set 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(train_img, train_mask_cat, test_size = 0.20, random_state = 0)
print(X_train.shape)
print(X_test.shape)

NameError: name 'train_test_split' is not defined

### Define the Dice Coefficient and Dice Coefficient Loss

In [ ]:
import tensorflow as tf

#Coefficients that is used during training.
def dice_coefficient(y_true, y_pred):
    """Computing the Dice Coefficient.
    Args:
        y_true:         the ground truth mask of shape (N,D,H,W,C), 
                        where N is the number of patches, D the depth of patches, H height of patches, 
                        W width of patches and C the number of channel
        y_pred:         the predicted mask by our mode of shape (N,D,H,W,C)
    Returns:
        dice:           Dice Coefficient computed, a scalar
    """
    smoothing_factor = 1
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)

#Loss function that might be used during the training.
def dice_coefficient_loss(y_true, y_pred):
    """Computing the Dice Coefficient.
      Args:
          y_true:         the ground truth mask of shape (N,D,H,W,C), 
                        where N is the number of patches, D the depth of patches, H height of patches, 
                        W width of patches and C the number of channel
          y_pred:         the predicted mask by our mode of shape (N,D,H,W,C)
      Returns:
          loss:           Dice Coefficient Loss computed, a scalar
    """
    y_true = tf.cast(y_pred, tf.float32)
    return 1 - dice_coefficient(y_true, y_pred)

### Training the model

In [ ]:
#Define parameters for our model.
#Here we use patches of size 16x128x128, so the input shape of our model needs to be the same. 
patch_size1 = 16
patch_size2 = 128
patch_size3 = 128
channels=1

LR = 0.0001
optim = keras.optimizers.Adam(LR)

#### Train with new model

In [ ]:
# Build our Model with the input size and number of classes chosen
model = build_unet((patch_size1,patch_size2,patch_size3,channels), n_classes)

# Compile the model with the optimizer and criterion which is the BCE Loss, as a metric we use the Dice Coefficient 
model.compile(optimizer = optim, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=dice_coefficient)

# Print the summary of our Model
print(model.summary())

#### Train a pre-trained model 

In [ ]:
from keras.models import load_model

# In case you want to train from a pre-trained model, run this cell.

# Load the pre-trained model and train from it 
# You need to specify another path file if the pre-trained model is not in the one one we specified, or simply add it to the /saved_models folder
model = load_model('/saved_models/3dunetmodel_leaky_bs4_16x128x128.h5', compile=False)
model.compile(optimizer = optim, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=dice_coefficient)

# Print the summary
print(model.summary())

In [ ]:
# Printing the model and inputs shape
print(model.input_shape)
print(X_train.shape)
print(model.output_shape)
print(y_train.shape)
print("-------------------")
print(X_train.max())  #Shpuld be 1 after scaling.

In [ ]:
# Fit the model
# You need to specify the batch size and the number of epochs. 

history=model.fit(X_train, 
          y_train,
          batch_size=4, 
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

### Saving the model for prediction

In [ ]:
# Save model for future use in the folder /saved_models
# In case you want to save it in another folder, you need to specify the file path.
model_path = '/saved_models/3dunetmodel_leaky_bs4_16x128x128.h5'
model.save(model_path)

### Plotting the training and validation IoU and loss at each epoch 

In [4]:
from matplotlib import pyplot as plt

##plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['dice_coefficient']
val_acc = history.history['val_dice_coefficient']

plt.plot(epochs, acc, 'y', label='Training Dice')
plt.plot(epochs, val_acc, 'r', label='Validation Dice')
plt.title('Training and validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.legend()
plt.show()

NameError: name 'history' is not defined

### Prediction with the model we trained

In [ ]:
import numpy as np

# Predict with the model trained
y_pred=model.predict(X_test)

#Predict on the test data
y_pred_argmax=np.argmax(y_pred, axis=4)
y_test_argmax = np.argmax(y_test, axis=4)

print(y_pred_argmax.shape)
print(y_test_argmax.shape)
print(np.unique(y_pred_argmax))

### Mean IoU 

In [ ]:
#Using built in keras function for IoU
#Only works on TF > 2.0
from keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test_argmax, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

### Testing random images 

In [ ]:
#Test some random images
import random

test_img_number = random.randint(0, len(X_test)-1)
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]

test_img_input=np.expand_dims(test_img, 0)


test_pred = my_model.predict(test_img_input)
test_prediction = np.argmax(test_pred, axis=4)[0,:,:,:]

ground_truth_argmax = np.argmax(ground_truth, axis=3)
print(ground_truth_argmax.shape)

#### Plotting the testing image, ground truth mask and the prediction 

In [ ]:
slice = random.randint(0, ground_truth_argmax.shape[0]-1)
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[slice,:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth_argmax[slice,:,:])
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction[slice,:,:])
plt.show()